In [ ]:
#########################################################################################
# Data Science Training Workshop: Using Jupyter Notebook and R (with a little Spark)
# Presenters: Leslie McIntosh, PhD, MPH; Connie Zabarovskaya, MITM; Lorinette Wirth, MPH
# 
# This script reads BREAST.txt data and parses into "original_data", which has 483,489 rows.
# It limits data to diagnosis years 1973-2002 and only 21 variables seen in data_dictionary 
# The subset is saved as data_sample.
# The script then creates the classification variable (survived) and "cleans" up the 
# rest of the variables for analysis. During the cleanup we also create one more 
# predictor variable based on Number of Positive Nodes (PN_PRESENT)
#########################################################################################

# SEER Research Data 1973-2013 -- ASCII Text Data:  Surveillance, Epidemiology, and End Results
# (SEER) Program (www.seer.cancer.gov) Research Data (1973-2013), National Cancer Institute,
# DCCPS, Surveillance Research Program, Surveillance Systems Branch, released April 2016,
# based on the November 2015 submission.
# see https://seer.cancer.gov/data/documentation.html

In [ ]:
# The three txt files below were prepared using Python, however, they can be prepared using another tool.
# Create vector of var names and widths from Python output then cobine as data frame 
names<- scan("name.txt", what="",sep=",")
width<-scan("width.txt",what="",sep=",")
start<-scan("start.txt",what="",sep=",")

start<-as.numeric(start)
width<-as.numeric(width)
names<-as.character(names)
info<-data.frame(names,start,width)
head(info)

In [ ]:
# THIS CELL WILL NOT BE RUN AT THE WORKSHOP, in order to save time.
# If you downloaded the BREAST.TXT file, feel free to uncomment the code.

# # read in breast.txt where each line is one record 
# read <- proc.time()
# raw_data <- readLines("BREAST.TXT")
# paste0("read time: ")
# proc.time() - read
# 
# trans <- proc.time()
# # function to split each line per start and end of each variable
# parseFixedWidthFile <- function(data, data_dictionary){
#   empty_df<- as.data.frame(matrix(0, ncol = 134, nrow = length(data)))
#   for (i in 1:nrow(data_dictionary)) {
#     newdata<-unlist(lapply(data,substr,data_dictionary$start[i],data_dictionary$start[i]+data_dictionary$width[i]-1))
#     empty_df[i]<-newdata
#   }
#   empty_df
# }
# 
# original_data <- parseFixedWidthFile(raw_data, info)
# # rename columns
# names(original_data) <- info$names
# paste0("transformation time: ")
# proc.time() - trans
# 
# save(original_data, file = "original_data.rdata" )

In [ ]:
load("original_data.rdata")

names(original_data)<-trimws(names(original_data))
# Checking for empty string values
paste0("Checking for empty string values")
table(original_data$YEAR_DX)
original_data$YEAR_DX <- as.numeric(original_data$YEAR_DX)
# limit to case dx between 1973 and 2002 
data_sample <- original_data[original_data$YEAR_DX >= 1973 & original_data$YEAR_DX <= 2002,]
# data_sample is 483489 rows

In [ ]:
keepVar<-c(
  "MAR_STAT"     ,
  "RACE1V"       ,
  "AGE_DX"       ,
  "PRIMSITE"     ,
  "GRADE"        ,
  "EOD10_SZ"     ,
  "EOD10_EX"     , 
  "EOD10_ND"     ,
  "EOD10_PN"     ,  
  "EOD10_NE"     , 
  "RADIATN"      , 
  "ADJAJCCSTG"   ,  
  "SEQ_NUM"      ,
  "HISTO3V"      ,
  "BEHO3V"       ,
  "SS_SURG"      ,
  "SURGPRIF"     ,
  "srv_time_mon" ,
  "STAT_REC"     ,
  "CODPUB"       ,
  "PUBCSNUM"     ,
  "REC_NO"       
) 
data_sample<-data_sample[,names(data_sample) %in% keepVar ]

save(data_sample,file="data_sample.rdata")

In [ ]:
# ON DOCKER: If you're not using conniez/all-spark-notebook image, uncomment the install command. 
# This needs to be run only once per container run (so the package is installed)
# suppressWarnings(suppressMessages(install.packages("dplyr", lib="/opt/conda/lib/R/library", repo="http://cran.us.r-project.org")))
# ON ANACONDA: If you haven't yet installed dplyr package, uncomment this command
# suppressWarnings(suppressMessages(install.packages("dplyr", repo="http://cran.us.r-project.org")))

In [ ]:
suppressWarnings(suppressMessages(library(dplyr)))
load("data_sample.rdata")

# Recoding Survivability Variable

# Cause of Death Codes downloaded with description and cancer, vs. non-cancer relatedness from here:
# https://seer.cancer.gov/codrecode/1969+_d09172004/index.txt

data_sample$srv_time_mon <- as.integer(data_sample$srv_time_mon)
# Cause of Death - Breast Cancer code is 26000
# Vital Status Recode (STAT_REC) Alive code is 1

# assign survived variable value 1 - survived, 0 - not survived, NA - to be ignored
# TO DO

# Display number of records where survived is not missing (NA)
paste0("Number of rows in data_sample")
# TO DO

# Display survivability breakdown
paste0("Distribution of survived variable")
# TO DO

# Drop NA values
# TO DO

In [ ]:
str(data_sample)

In [ ]:
# Identifying and Recoding Missing Values for Predictor Variables

# Tumor Size
# Remove values of tumor size >200mm in accordance with Paper by Delen et al.
# Tumor size in Paper by Bellaachia and Guven: mean=20, sd=16, range=0-200
# Checking for empty string values
sort(unique(data_sample$EOD10_SZ))
# There are values consisting of 3 spaces
data_sample$EOD10_SZ[data_sample$EOD10_SZ %in% c('   ')] <- NA
data_sample$EOD10_SZ <- as.integer(data_sample$EOD10_SZ)
data_sample$EOD10_SZ[data_sample$EOD10_SZ > 200]<-NA
EOD10_SZrange <- range(data_sample$EOD10_SZ,na.rm = T)
paste0("Range: ",paste(EOD10_SZrange, collapse = "-"))
EOD10_SZmean <- round(mean(data_sample$EOD10_SZ,na.rm = T), 2)
paste0("Mean: ",EOD10_SZmean)
EOD10_SZsd <- round(sd(data_sample$EOD10_SZ,na.rm = T),2)
paste0("Standard Deviation: ",EOD10_SZsd)
paste0("Number of records with missing or invalid tumor size")
table(is.na(data_sample$EOD10_SZ))

In [ ]:
# Age
# Age in Paper by Bellaachia and Guven:mean=58, sd=13, range=10-110
# Checking for empty string values
sort(unique(data_sample$AGE_DX))
data_sample$AGE_DX[data_sample$AGE_DX %in% c(999, ' ')] <- NA
data_sample$AGE_DX<- as.integer(data_sample$AGE_DX)
AGE_DXrange <- range(data_sample$AGE_DX,na.rm = T)
paste0("Range: ",paste(AGE_DXrange, collapse = "-"))
AGE_DXmean <- round(mean(data_sample$AGE_DX,na.rm = T),2)
paste0("Mean: ",AGE_DXmean)
AGE_DXsd <- round(sd(data_sample$AGE_DX,na.rm = T),2)
paste0("Standard Deviation: ",AGE_DXsd)
paste0("Number of records with missing or invalid AGE_DX")
table(is.na(data_sample$AGE_DX))

In [ ]:
# Presense of Positive Nodes
# This is a variable calculated based on No. of Positive Nodes
# See No. of Positive Nodes codes for details
# 00, 95 - code as 0, "No positive nodes present"
# 01-90 - code as 1, "Positive nodes present"
# 98, 99 - code as NA, "No nodes were examined OR Unknown whether nodes are positive" 
data_sample <- mutate(data_sample, PN_PRESENT = ifelse(EOD10_PN %in% c("95","00"), 0,
                                                ifelse(EOD10_PN %in% c("98","99"), NA, 1)))
data_sample$PN_PRESENT <- as.factor(data_sample$PN_PRESENT)
table(data_sample$PN_PRESENT)
PN_PRESENTlev <- nlevels(data_sample$PN_PRESENT)
paste0("Number of records with missing or invalid PN_PRESENT")
table(is.na(data_sample$PN_PRESENT))

In [ ]:
# No. of Positive Nodes
# This field has some weird values
# Code Description
# 00 All nodes examined are negative
# 01-89 Exact number of nodes positive
# 90 90 or more nodes are positive
# 95 Positive aspiration of lymph node(s) was performed
# 97 Positive nodes are documented, but number is unspecified
# 98 No nodes were examined
# 99 Unknown whether nodes are positive; not applicable; not
# stated in patient record

# Checking for empty string values
sort(unique(data_sample$EOD10_PN))
# Based on that dictionary, consider renaming all values >90 to be NA
data_sample$EOD10_PN <- as.integer(data_sample$EOD10_PN)
# this also creates NAs if value was empty string
table(data_sample$EOD10_PN[data_sample$EOD10_PN > 90])
data_sample$EOD10_PN[data_sample$EOD10_PN > 90]<-NA
EOD10_PNrange <- range(data_sample$EOD10_PN,na.rm=T)
paste0("Range: ", paste(EOD10_PNrange, collapse="-"))
EOD10_PNmean <- round(mean(data_sample$EOD10_PN,na.rm=T),2)
paste0("Mean: ",EOD10_PNmean)
EOD10_PNsd <- round(sd(data_sample$EOD10_PN,na.rm=T),2)
paste0("Standard Deviation: ",EOD10_PNsd)
paste0("Number of records with missing or invalid EOD10_PN")
table(is.na(data_sample$EOD10_PN))

In [ ]:
# Number of Nodes Examined (Number of Nodes)
# Code Description
# 00 No nodes were examined
# 01-89 Exact number of nodes examined
# 90 90 or more nodes were examined
# 95 No regional nodes were removed, but aspiration of regional
# nodes was performed
# 96 Regional lymph node removal was documented as a
# sampling, and the number of nodes is unknown/not stated
# 97 Regional lymph node removal was documented as a
# dissection, and the number of nodes is unknown/not stated
# 98 Regional lymph nodes were surgically removed, but the
# number of lymph nodes is unknown/not stated and not
# documented as a sampling or dissection; nodes were
# examined, but the number is unknown
# 99 Unknown whether nodes were examined; not applicable or
# negative; not stated in patient record

# Checking for empty string values
sort(unique(data_sample$EOD10_NE))
# Based on that dictionary, consider renaming all values >90 to be NA
data_sample$EOD10_NE<-as.integer(data_sample$EOD10_NE)
table(data_sample$EOD10_NE[data_sample$EOD10_NE > 90])
data_sample$EOD10_NE[data_sample$EOD10_NE > 90]<-NA
EOD10_NErange <- range(data_sample$EOD10_NE,na.rm=T)
paste0("Range: ",paste(EOD10_NErange, collapse="-"))
EOD10_NEmean <- round(mean(data_sample$EOD10_NE,na.rm=T), 2)
paste0("Mean: ",EOD10_NEmean)
EOD10_NEsd <- round(sd(data_sample$EOD10_NE,na.rm=T), 2)
paste0("Standard Deviation: ",EOD10_NEsd)
paste0("Number of records with missing or invalid EOD10_NE")
table(is.na(data_sample$EOD10_NE))

In [ ]:
# Number of primaries
# In Situ/Malignant as Federally Required based on Diagnosis Year
# Code Description
# 00 One primary only in the patient's lifetime
# 01 First of two or more primaries
# 02 Second of two or more primaries
# .. (Actual number of this primary)
# 41 Forty-first of forty-one or more primaries
# 99 Unspecified or unknown sequence number of Federally required in situ or malignant
# tumors. Sequence number 99 can be used if there is a malignant tumor and its sequence
# number is unknown. (If there is known to be more than one malignant tumor, then the
#                     tumors must be sequenced.)
# Non-malignant Tumor as Federally Required based on Diagnosis Year
# Code Description
# 60 Only one non-malignant tumor or central registry-defined neoplasm
# 61 First of two or more non-malignant tumors or central registry-defined neoplasms
# 62 Second of two or more non-malignant tumors or central registry-defined neoplasms
# .. ..
# 87 Twenty-seventh of twenty-seven
# 88 Unspecified or unknown sequence number of non-malignant tumor or central-registry
# defined neoplasms. (Sequence number 88 can be used if there is a non-malignant tumor
#                     and its sequence number is unknown. If there is known to be more than one nonmalignant
#                     tumor, then the tumors must be sequenced.)

# Checking for empty string values
as.data.frame(table(data_sample$SEQ_NUM))
# should not convert to numeric as is, because 0 actually means a single primary.
# perhaps leave this as character var, or convert to factor, if decision tree requires factors (which it sometimes does)
# however, based on Paper by Bellaachia and Guven, perhaps we could rename 0 to 1
data_sample$SEQ_NUM[data_sample$SEQ_NUM %in% c(99,88,' ')]<-NA
data_sample$SEQ_NUM<-as.integer(data_sample$SEQ_NUM)
data_sample$SEQ_NUM[data_sample$SEQ_NUM == 0] <- 1
SEQ_NUMrange <- range(data_sample$SEQ_NUM,na.rm=T)
paste0("Range: ",paste(SEQ_NUMrange, collapse = "-"))
SEQ_NUMmean <- round(mean(data_sample$SEQ_NUM,na.rm=T),2)
paste0("Mean: ",SEQ_NUMmean)
SEQ_NUMsd <- round(sd(data_sample$SEQ_NUM,na.rm=T),2)
paste0("Standard Deviation: ",SEQ_NUMsd)
paste0("Number of records with missing or invalid Number of primaries")
table(is.na(data_sample$SEQ_NUM))

In [ ]:
# Marital Status
# Code Description
# 1 Single (never married)
# 2 Married (including common law)
# 3 Separated
# 4 Divorced
# 5 Widowed
# 6 Unmarried or domestic partner (same sex or opposite sex or
#                                  unregistered)
# 9 Unknown

# Use sort and unique to check for empty string values
# TO DO

# Assign NA where MAR_STAT equals 9
# TO DO

# Assign 1 where MAR_STAT is 1 or 6 (combine the two categories together)
# Unmarried but living with part combined =with single
# TO DO


# Convert MAR_STAT to a factor
# TO DO

# Display distribution of values in MAR_STAT
# TO DO

# Create variable MAR_STATlev to store the number of levels in MAR_STAT variable
# TO DO
paste0("Number of factor levels: ",MAR_STATlev)
paste0("Number of records with missing or invalid MAR_STAT")

# Display breakdown of NA and non-missing values in MAR_STAT
# TO DO


In [ ]:
# Race
# Checking for empty string values
sort(unique(data_sample$RACE1V))

# We combine races that have small representation (< 300) into category "Other"
table(as.factor(data_sample$RACE1V))
race_dist <- as.data.frame(table(as.factor(data_sample$RACE1V)))
data_sample$RACE1V[data_sample$RACE1V %in% race_dist[race_dist$Freq < 300, "Var1"]]<-98
data_sample$RACE1V[data_sample$RACE1V %in% c(99)]<-NA
data_sample$RACE1V<-as.factor(data_sample$RACE1V)
table(data_sample$RACE1V)
RACE1Vlev <- nlevels(data_sample$RACE1V)
paste0("Number of factor levels: ",RACE1Vlev)
table(is.na(data_sample$RACE1V))

In [ ]:
# Primary Site code
# Checking for empty string values
sort(unique(data_sample$PRIMSITE))
data_sample$PRIMSITE<-as.factor(data_sample$PRIMSITE)
PRIMSITElev <- nlevels(data_sample$PRIMSITE)
paste0("Number of factor levels: ",PRIMSITElev)
table(data_sample$PRIMSITE)
table(is.na(data_sample$PRIMSITE))

In [ ]:
# Histologic type
# Checking for empty string values
# Recode values to reduce number of levels
# Based on these sources, we used the following mapping
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2361680/
# http://jamanetwork.com/journals/jamainternalmedicine/fullarticle/216158
# 
# ".... grouped according to the histologic categories of their tumors as defined 
# by the International Classification of Diseases for Oncology (ICD-O) codes: 
# "0001" - ductal (ICD-O code 8500), 
# "0002" - lobular (8520 and 8522), 
# "0003" - mucinous (8480), 
# "0004" - comedocarcinoma (8501), 
# "0005" - medullary (8510), 
# "0006" - tubular (8211 and 8201), 
# "0007" - and papillary (8050 and 8503)"
# "0008" - Code all others as "Other."

table(data_sample$HISTO3V)
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8500")] <- "0001"
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8520","8522")] <- "0002"
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8480")] <- "0003"
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8501")] <- "0004"
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8510")] <- "0005"
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8211","8201")] <- "0006" 
data_sample$HISTO3V[data_sample$HISTO3V %in% c("8050","8503")] <- "0007"
data_sample$HISTO3V[!data_sample$HISTO3V %in% c("0001","0002","0003","0004","0005","0006","0007")] <- "0008"
data_sample$HISTO3V<-as.factor(data_sample$HISTO3V)
HISTO3Vlev <- nlevels(data_sample$HISTO3V)
paste0("Number of factor levels: ",HISTO3Vlev)
table(is.na(data_sample$HISTO3V))

In [ ]:
# Behavior
# Checking for empty string values
sort(unique(data_sample$BEHO3V))
data_sample$BEHO3V<-as.factor(data_sample$BEHO3V)
BEHO3Vlev <- nlevels(data_sample$BEHO3V)
paste0("Number of factor levels: ",BEHO3Vlev)
table(data_sample$BEHO3V)
table(is.na(data_sample$BEHO3V))

In [ ]:
# Grade
# Code Description
# 1 Grade I; grade i; grade 1; well differentiated; differentiated, NOS
# 2 Grade II; grade ii; grade 2; moderately differentiated; moderately differentiated; intermediate differentiation
# 3 Grade III; grade iii; grade 3; poorly differentiated; differentiated
# 4 Grade IV; grade iv; grade 4; undifferentiated; anaplastic
# 5 T-cell; T-precursor
# 6 B-cell; Pre-B; B-Precursor
# 7 Null cell; Non T-non B;
# 8 N K cell (natural killer cell)
# 9 cell type not determined, not stated or not applicable

# Checking for empty string values
sort(unique(data_sample$GRADE))
data_sample$GRADE[data_sample$GRADE %in% c(9)]<-NA
data_sample$GRADE<-as.factor(data_sample$GRADE)
GRADElev <- nlevels(data_sample$GRADE)
paste0("Number of factor levels: ",GRADElev)
table(data_sample$GRADE)
table(is.na(data_sample$GRADE))

In [ ]:
# Extension of Tumor
# Allowable values = 00-99.
# Recode child nodes to parent nodes according to this source
# Page 120 of https://seer.cancer.gov/archive/manuals/EOD10Dig.pub.pdf
# Checking for empty string values
sort(unique(data_sample$EOD10_EX))
# There are multiple 2-space values
data_sample$EOD10_EX_num <- as.numeric(data_sample$EOD10_EX)
data_sample <- mutate(data_sample, 
                       EOD10_EX_str = ifelse(EOD10_EX_num >= 10 & EOD10_EX_num < 20, "10",
                                      ifelse(EOD10_EX_num >= 20 & EOD10_EX_num < 30, "20",
                                      ifelse(EOD10_EX_num >= 30 & EOD10_EX_num < 40, "30",
                                      ifelse(EOD10_EX_num == 0, "00", 
                                      ifelse(EOD10_EX_num == 5, "05",EOD10_EX_num))))))
data_sample$EOD10_EX <- data_sample$EOD10_EX_str
data_sample$EOD10_EX[data_sample$EOD10_EX %in% c(99)]<-NA
data_sample$EOD10_EX<-as.factor(data_sample$EOD10_EX)
EOD10_EXlev <- nlevels(data_sample$EOD10_EX)
paste0("Number of factor levels: ",EOD10_EXlev)
table(is.na(data_sample$EOD10_EX))

In [ ]:
# Lymph node involvement
# Checking for empty string values
# Allowable values = 0-9.
# 0 No lymph node involvement
# 9 UNKNOWN; not stated
sort(unique(data_sample$EOD10_ND))
data_sample$EOD10_ND[data_sample$EOD10_ND %in% c(9,' ')]<-NA
data_sample$EOD10_ND<-as.factor(data_sample$EOD10_ND)
EOD10_NDlev <- nlevels(data_sample$EOD10_ND)
paste0("Number of factor levels: ",EOD10_NDlev)
table(is.na(data_sample$EOD10_ND))

In [ ]:
# Site Specific Surgery Code....priotize SURGPRIF
table(data_sample$SS_SURG)
# Considering https://seer.cancer.gov/seerstat/variables/seer/surgery/
# Code 	Description
# 00 	No surgical procedure
# 09 	Unknown if surgery done
# 90 	Surgery, NOS
# See other code definitions here: https://seer.cancer.gov/archive/manuals/historic/AppendD.pdf


In [ ]:
# SURGPRIF - Surgery Primary Syte
# Code Description
# 00 None; no surgical procedure of primary site; diagnosed at
# autopsy only
# 10-19 Site-specific codes. Tumor destruction; no pathologic
# specimen or unknown whether there is a pathologic
# specimen
# 20-80 Site-specific codes. Resection; pathologic specimen
# 90 Surgery, NOS. A surgical procedure to the primary site
# was done, but no information on the type of surgical
# procedure is provided.
# 98 Special codes for hematopoietic, reticuloendothelial,
# immunoproliferative, myeloproliferative diseases; illdefined
# sites; and unknown primaries (See site-specific
#                               codes for the sites and histologies), except death
# certificate only
# 99 Unknown if surgery performed; death certificate only
# See detailed code descriptions here: https://seer.cancer.gov/archive/manuals/AppendC.pdf


In [ ]:
# Radiation
# Code  Description
# 0  None; diagnosed at autopsy
# 1  Beam radiation
# 2  Radioactive implants
# 3  Radioisotopes
# 4  Combination of 1 with 2 or 3
# 5  Radiation, NOS - method or source not specified
# 6  Other radiation (1973-1987 cases only)
# 7  Patient or patient's guardian refused radiation therapy
# 8  Radiation recommended, unknown if administered
# 9  Unknown if radiation administered

# Checking for empty string values
sort(unique(data_sample$RADIATN))
data_sample$RADIATN[data_sample$RADIATN %in% c(9)]<-NA
data_sample$RADIATN<-as.factor((data_sample$RADIATN))
RADIATNlev <- nlevels(data_sample$RADIATN)
paste0("Number of factor levels: ",RADIATNlev)
table(is.na(data_sample$RADIATN))

In [ ]:
# Stage of Cancer
# Code Description
# 88 N/A
# 90 OCCULT
# 99 UNK Stage
# Checking for empty string values
sort(unique(data_sample$ADJAJCCSTG))
# There are values that have 2 blank spaces (not 1)
data_sample$ADJAJCCSTG[data_sample$ADJAJCCSTG %in% c(88,'  ',99)]<-NA
data_sample$ADJAJCCSTG<-as.factor(data_sample$ADJAJCCSTG)
ADJAJCCSTGlev <- nlevels(data_sample$ADJAJCCSTG)
paste0("Number of factor levels: ",ADJAJCCSTGlev) 
table(is.na(data_sample$ADJAJCCSTG))

In [ ]:
# save this version of data_sample as clean_data_sample if needed to load later
clean_data_sample <- data_sample
save(clean_data_sample, file = "clean_data_sample.rdata")

In [ ]:
load("clean_data_sample.rdata")

# data_sample <- clean_data_sample

# Recode SS_SURG and SURGPRIF to a common Site-Specific Surgery Code
ss_surg_surgprif_map <- read.csv("ss_surg_surgprif_map.csv", colClasses=c("character","character"))
str(ss_surg_surgprif_map)

data_sample <- merge(data_sample, ss_surg_surgprif_map, by.x = "SS_SURG", by.y = "CODE",
                     all.x = TRUE)
data_sample <- merge(data_sample, ss_surg_surgprif_map, by.x = "SURGPRIF", by.y = "CODE",
                     all.x = TRUE)
# recoding into one var
data_sample <- mutate(data_sample, SURGCODE = ifelse(SURGCODE.x=="", SURGCODE.y, SURGCODE.x))
# recoding "Unknown" codes into NAs
# 99 - Uknown if surgery was done
data_sample$SURGCODE[data_sample$SURGCODE %in% c("99")] <- NA

data_sample$SURGCODE <- as.factor(data_sample$SURGCODE)
SURGCODElev <- nlevels(data_sample$SURGCODE)

In [ ]:
# Dataset with NAs included
full_surgcode_data_sample <- data_sample[,c(
  "MAR_STAT"     ,
  "RACE1V"       ,
  "AGE_DX"       ,
  "PRIMSITE"     ,
  "GRADE"        ,
  "EOD10_SZ"     ,
  "EOD10_EX"     , 
  "EOD10_ND"     ,
  "EOD10_PN"     ,
  "PN_PRESENT"   ,
  "EOD10_NE"     , 
  "RADIATN"      , 
  "ADJAJCCSTG"   ,  
  "SEQ_NUM"      ,
  "HISTO3V"      ,
  "BEHO3V"       ,
  "SURGCODE"      ,
  "survived"
)]
paste0("Number of rows in Dataset with NAs included")
nrow(full_surgcode_data_sample)
save(full_surgcode_data_sample, file = "full_surgcode_data_sample.rdata")

In [ ]:
#####################################################################################
# Dataset with complete cases (no NAs included)
clean_surgcode_data_sample<-na.omit(data_sample[,c(
  "MAR_STAT"     ,
  "RACE1V"       ,
  "AGE_DX"       ,
  "PRIMSITE"     ,
  "GRADE"        ,
  "EOD10_SZ"     ,
  "EOD10_EX"     , 
  "EOD10_ND"     ,
  "EOD10_PN"     ,
  "PN_PRESENT"   ,
  "EOD10_NE"     , 
  "RADIATN"      , 
  "ADJAJCCSTG"   ,  
  "SEQ_NUM"      ,
  "HISTO3V"      ,
  "BEHO3V"       ,
  "SURGCODE"      ,
  "survived"
)])

paste0("Number of rows in Dataset with complete cases")
nrow(clean_surgcode_data_sample)
save(clean_surgcode_data_sample, file = "clean_surgcode_data_sample.rdata")

In [ ]:
# Table 1 - describing the full dataset
data.frame(Nominal_Variable_Name = c("Race", "Marital status", "Primary site code", "Histologic type", 
                                    "Behavior code", "Grade", "Extension of tumor", "Lymph node involvement",
                                    "Presense of positive nodes", "Site specific surgery code", "Radiation", 
                                     "Stage of Cancer"),
          Number_of_Distinct_Values = c(RACE1Vlev, MAR_STATlev, PRIMSITElev,HISTO3Vlev, BEHO3Vlev,
                                       GRADElev, EOD10_EXlev, EOD10_NDlev,PN_PRESENTlev, 
                                        SURGCODElev,RADIATNlev, ADJAJCCSTGlev))

data.frame(Numeric_Variable_Name = c("Age", "Tumor Size", "No of positive nodes", "Number of nodes", "Number of primaries"),
           Mean = c(AGE_DXmean, EOD10_SZmean, EOD10_PNmean, EOD10_NEmean, SEQ_NUMmean),
           StdDev = c(AGE_DXsd, EOD10_SZsd, EOD10_PNsd, EOD10_NEsd, SEQ_NUMsd),
           Range = c(paste(AGE_DXrange, collapse = "-"), paste(EOD10_SZrange, collapse = "-"),
                    paste(EOD10_PNrange, collapse = "-"),paste(EOD10_NErange, collapse="-"),
                    paste(SEQ_NUMrange, collapse="-"))
          )